<a href="https://colab.research.google.com/github/xuziyue/tensorflow-models/blob/main/cnn_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>